In [10]:
import pickle
import os

import matplotlib.pyplot as plt
import networkx as nx
from networkx.drawing.nx_agraph import graphviz_layout, to_agraph
import pygraphviz
%matplotlib inline

In [12]:
# initialize graph
G = nx.DiGraph()

circuit_path = "../simple_circuit.pkl"
circuit_figure_filename = os.path.basename(circuit_path).split(".pkl")[0] + ".png"
circuit_figure_path = os.path.join("../figures/circuits", circuit_figure_filename)

# load dictionary
with open(circuit_path, "rb") as circuit_pickle:
    circuit = pickle.load(circuit_pickle)

In [13]:
G.graph['node'] = {'shape': 'box', 'style': 'rounded'}
G.graph['graph'] = {'rankdir': 'BT'}
textcolor = "black"     # easy to change in case we use colors where white text will look better

for parent in circuit:
    if parent not in G.nodes:
        if "mlp" in parent:
            nodecolor = "#e5ad76"   # camel
        elif "attn" in parent:
            nodecolor = "#5baf7e"   # kelly green
        elif "resid" in parent:
            nodecolor = "#88a7cd"   # pastel steel blue
        else:
            nodecolor = "white"

        if parent != "y":
            layer, feat_idx, component = parent.split("_")
            nodename = f"{component}_{layer}/{feat_idx}"
        else:
            nodename = "y"
        G.add_node(parent, label=nodename, style="rounded,filled", fillcolor=nodecolor, fontcolor=textcolor)
    for child in circuit[parent]:
        child_name, child_effect = child
        if child_name not in G.nodes:
            if "mlp" in child_name:
                nodecolor = "#e5ad76"
            elif "attn" in child_name:
                nodecolor = "#5baf7e"
            elif "resid" in child_name:
                nodecolor = "#88a7cd"
            else:
                nodecolor = "white"

            layer, feat_idx, component = child_name.split("_")
            nodename = f"{component}_{layer}/{feat_idx}"
            G.add_node(child_name, label=nodename, style="rounded,filled", fillcolor=nodecolor, fontcolor=textcolor)
        G.add_edge(child_name, parent)

A = to_agraph(G)
A.layout('dot')

if not os.path.exists("../figures/circuits/"):
    os.makedirs("../figures/circuits/")
A.draw(circuit_figure_path)

AttributeError: module 'os' has no attribute 'exists'